In [28]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import json
import os
import re

In [46]:
#Game Data from main understat page
result = requests.get('https://understat.com/league/EPL')
src = result.content
soup = BeautifulSoup(src, 'html.parser')
Misa = soup.find_all("script")
Game_Table_Player_data = []
for bruh in Misa[1:4]:
    bruh = str(bruh)
    s_1 = bruh.find('\'')
    s_2 = bruh.rfind('\'')
    bruh = bruh[s_1:s_2 + 1]
    Game_Table_Player_data.append(bruh)
Game_ID = []
is_result = []
Home = []
Away = []
date_time = []
Home_Goals = []
Away_Goals = []
xG_Home = []
xG_Away = []
Prob_Home_Win = []
Prob_Draw = []
Prob_Away_Win = []
Game_data = json.loads(Game_Table_Player_data[0].encode('ascii').decode('unicode-escape')[1:-1])

#Function w domain/ range as ids/ names
T_ID = []
T_name = []
for i in range(0, len(list(Game_data))):
    T_ID.append(Game_data[i]['h']['id'])
    T_name.append(Game_data[i]['h']['title'])
T_dict = { T_ID[i] : T_name[i] for i in range(0, len(T_name))}
def id_to_name(ID):
    return T_dict[ID]

#Putting the game data into a dataframe
for Game in Game_data:
    Game['h'] = Game['h']['title']
    Game['a'] = Game['a']['title']
    Game['Home Goals'] = Game['goals']['h']
    Game['Away Goals'] = Game['goals']['a']
    Game['xG Home'] = Game['xG']['h']
    Game['xG Away'] = Game['xG']['a']
    if ('forecast' in list(Game.keys())):
        Game['Probability Home Win'] = Game['forecast']['w']
        Game['Probability Draw'] = Game['forecast']['d']
        Game['Probability Away Win'] = Game['forecast']['l']
        del Game['forecast']
        Prob_Home_Win.append(list(Game.values())[11])
        Prob_Draw.append(list(Game.values())[12])
        Prob_Away_Win.append(list(Game.values())[13])
    else:
        Prob_Home_Win.append('N/A')
        Prob_Draw.append('N/A')
        Prob_Away_Win.append('N/A')
    del Game['goals']
    del Game['xG']
    Game_ID.append(list(Game.values())[0])
    is_result.append(list(Game.values())[1])
    Home.append(list(Game.values())[2])
    Away.append(list(Game.values())[3])
    date_time.append(list(Game.values())[4])
    Home_Goals.append(list(Game.values())[5])
    Away_Goals.append(list(Game.values())[6])
    xG_Home.append(list(Game.values())[7])
    xG_Away.append(list(Game.values())[8])
Game_Data = pd.DataFrame({'id': Game_ID, 'Is Result?': is_result, 'Home Team': Home, 
                                              'Away Team': Away, 'Home Goals': Home_Goals, 'Away Goals': Away_Goals,
                                          'xG Home' : xG_Home, 'xG Away': xG_Away, 'Probability Home Win': Prob_Home_Win,
                                             'Probability Draw': Prob_Draw, 'Probability Away Win': Prob_Away_Win})

#Player Data from main Understat page
Player_ID = []
Player_name = []
Games = []
playtime_minutes = []
Goals = []
xG = []
Assists = []
xA = []
shots = []
key_passes = []
yellow_cards = []
red_cards = []
positions = []
team = []
nonpen_goals = []
nonpen_xG = []
xG_Chain = []
xG_Buildup = []
Player_data  = json.loads(Game_Table_Player_data[2].encode('ascii').decode('unicode-escape')[1:-1])
for Game in Player_data:
    Player_ID.append(list(Game.values())[0])
    Player_name.append(list(Game.values())[1])
    Games.append(list(Game.values())[2])
    playtime_minutes.append(list(Game.values())[3])
    Goals.append(list(Game.values())[4])
    xG.append(list(Game.values())[5])
    Assists.append(list(Game.values())[6])
    xA.append(list(Game.values())[7])
    shots.append(list(Game.values())[8])
    key_passes.append(list(Game.values())[9])
    yellow_cards.append(list(Game.values())[10])
    red_cards.append(list(Game.values())[11])
    positions.append(list(Game.values())[12])
    team.append(list(Game.values())[13])
    nonpen_goals.append(list(Game.values())[14])
    nonpen_xG.append(list(Game.values())[15])
    xG_Chain.append(list(Game.values())[16])
    xG_Buildup.append(list(Game.values())[17])
PL_Player_Data_2021 = pd.DataFrame({'id': Player_ID, 'Name': Player_name, 'Games Played': Games, 
                                  'Minutes Played': playtime_minutes, 'Goals': Goals, 'xG': xG,
                              'Assists' : Assists, 'xA': xA, 'Shots': shots, 'Key Passes': key_passes,
                                 'Yellow Cards': yellow_cards, 'Red Cards': red_cards, 'Positions': positions,
                                   'Team': team, 'Non Penalty Goals': nonpen_goals, 'Non Penalty xG': nonpen_xG,
                                   'xG Chain': xG_Chain, 'xG Buildup': xG_Buildup})
#Id/ name for game correspondence and adding game name to the dataframe
Game_name = []
for i in range(0, len(Home)):
    game = Home[i] + ' Vs ' + Away[i]
    Game_name.append(game)
Game_Data['Game'] = Game_name
G_dict = {Game_ID[i] : Game_name[i] for i in range(0, len(Game_name))}
def Game_id_to_name(ID):
    return G_dict[ID]

In [50]:
print(Game_Data)

        id  Is Result?                Home Team          Away Team Home Goals  \
0    14086        True                   Fulham            Arsenal          0   
1    14087        True           Crystal Palace        Southampton          1   
2    14090        True                Liverpool              Leeds          4   
3    14091        True                 West Ham   Newcastle United          0   
4    14092        True     West Bromwich Albion          Leicester          0   
..     ...         ...                      ...                ...        ...   
375  14810        True                Liverpool     Crystal Palace          2   
376  14811        True          Manchester City            Everton          5   
377  14812        True         Sheffield United            Burnley          1   
378  14813        True                 West Ham        Southampton          3   
379  14814        True  Wolverhampton Wanderers  Manchester United          1   

    Away Goals   xG Home   

In [55]:
shot_data = []
match_data = []
player_data = []
for ID in Game_ID:
    result = requests.get('https://understat.com/match/' + ID)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    Misa = soup.find_all("script")
    List = []
    for bruh in Misa[1:3]:
        bruh = str(bruh)
        Jeff = [m.start() for m in re.finditer('\'', bruh)]
    
        if len(Jeff) > 2:
            bruh_1 = bruh[Jeff[0]: Jeff[1] + 1]
            bruh_2 = bruh[Jeff[2]: Jeff[3] + 1]
            List.append(bruh_1)
            List.append(bruh_2)
        else:
            bruh = bruh[Jeff[0]: Jeff[1] + 1]
            List.append(bruh)
    #defining what each list gives us
    Shot_data = json.loads(List[0].encode('ascii').decode('unicode-escape')[1:-1])
    ID = []
    minute = []
    result = []
    X = []
    Y = []
    xG = []
    player = []
    h_a = []
    player_id = []
    situation = []
    season = []
    shotType = []
    match_id = []
    h_team = []
    a_team = []
    player_assisted = []
    lastAction = []
    for loc in ['h', 'a']:
        for i in range(0, len(Shot_data[loc])):
            ID.append(Shot_data[loc][i]['id'])
            minute.append(Shot_data[loc][i]['minute'])
            result.append(Shot_data[loc][i]['result'])
            X.append(Shot_data[loc][i]['X'])
            Y.append(Shot_data[loc][i]['Y'])
            xG.append(Shot_data[loc][i]['xG'])
            player.append(Shot_data[loc][i]['player'])
            h_a.append(Shot_data[loc][i]['h_a'])
            player_id.append(Shot_data[loc][i]['player_id'])
            situation.append(Shot_data[loc][i]['situation'])
            season.append(Shot_data[loc][i]['season'])
            shotType.append(Shot_data[loc][i]['shotType'])
            match_id.append(Shot_data[loc][i]['match_id'])
            h_team.append(Shot_data[loc][i]['h_team'])
            a_team.append(Shot_data[loc][i]['a_team'])
            player_assisted.append(Shot_data[loc][i]['player_assisted'])
            lastAction.append(Shot_data[loc][i]['lastAction'])
    Shot_df = pd.DataFrame({'ID': ID, 'Minute' : minute, 'Result' : result, 'X_coord': X, 'Y_coord': Y, 'xG' : xG, 
                            'Player name': player, 'Home/Away': h_a, 'Player ID' : player_id, 'Situation' : situation,
                           'Season' : season, 'Shot Type': shotType, 'Match ID' : match_id, 'Home Team': h_team, 
                           'Away Team': a_team, 'Player Assisted': player_assisted, 'Last Action': lastAction})
    Shot_df['Game'] = Shot_df['Home Team'] + ' Vs ' + Shot_df['Away Team']
    shot_data.append(Shot_df)
    #Using Match Data, need to continue it outside the loop
    Match_data = json.loads(List[1].encode('ascii').decode('unicode-escape')[1:-1])
    
    df_match = pd.DataFrame({'Home Shots' : Match_data['h_shot'], 'Away Shots' : Match_data['a_shot'], 
                'Home Shots on Target': Match_data['h_shotOnTarget'], 'Away Shots on Target' : Match_data['a_shotOnTarget']}
                           , index = [0])
    match_data.append(df_match)
    #Using Player Data
    Player_data = json.loads(List[2].encode('ascii').decode('unicode-escape')[1:-1])
    ID = []
    goals = []
    own_goals = []
    shots = []
    xG = []
    time = []
    player_id = []
    team_id = []
    position = []
    name = []
    home_away = []
    yellow_cards = []
    red_cards = []
    roster_in = []
    roster_out = []
    key_passes = []
    assists = []
    xA = []
    xG_chain = []
    xG_Buildup = []
    position_order = []
    h_a = ['h', 'a']
    for loc in h_a:
        for player in list(Player_data[loc].keys()):
            ID.append(list(Player_data[loc][player].values())[0])
            goals.append(list(Player_data[loc][player].values())[1])
            own_goals.append(list(Player_data[loc][player].values())[2])
            shots.append(list(Player_data[loc][player].values())[3])
            xG.append(list(Player_data[loc][player].values())[4])
            time.append(list(Player_data[loc][player].values())[5])
            player_id.append(list(Player_data[loc][player].values())[6])
            team_id.append(list(Player_data[loc][player].values())[7])
            position.append(list(Player_data[loc][player].values())[8])
            name.append(list(Player_data[loc][player].values())[9])
            home_away.append(list(Player_data[loc][player].values())[10])
            yellow_cards.append(list(Player_data[loc][player].values())[11])
            red_cards.append(list(Player_data[loc][player].values())[12])
            roster_in.append(list(Player_data[loc][player].values())[13])
            roster_out.append(list(Player_data[loc][player].values())[14])
            key_passes.append(list(Player_data[loc][player].values())[15])
            assists.append(list(Player_data[loc][player].values())[16])
            xA.append(list(Player_data[loc][player].values())[17])
            xG_chain.append(list(Player_data[loc][player].values())[18])
            xG_Buildup.append(list(Player_data[loc][player].values())[19])
            position_order.append(list(Player_data[loc][player].values())[20])
    df = pd.DataFrame({'Id' : ID, 'Goals' : goals, 'Own Goals': own_goals, 'Shots' : shots, 'xG': xG, 'time' : time, 
                           'Player ID' : player_id, 'Team Id' : team_id, 'Position' : position, 'Player Name' : name, 'Home/Away':
                           home_away, 'Yellow Cards': yellow_cards, 'Red Cards': red_cards, 'Roster In': roster_in, 
                           'Roster Out':roster_out,'Key passes' : key_passes, 'Assists' : assists, 'xA': xA, 'xG Chain': xG_chain,
                           'xG Buildup' : xG_Buildup, 'Position order' : position_order})
    player_data.append(df)
Season_Player_Data = pd.concat(player_data, ignore_index = True, keys=Game_name)
Season_Player_Data['Team Name'] = Season_Shot_Data['Team Id'].apply(id_to_name)
Season_Match_Data = pd.concat(match_data, ignore_index = True, keys = Game_name)
Season_Shot_Data = pd.concat(shot_data,ignore_index = True, keys = Game_name)

In [60]:
Season_Player_Data = pd.concat(player_data, ignore_index = True, keys=Game_name)
#Season_Player_Data['Team Name'] = Season_Shot_Data['Team Id'].apply(id_to_name)
Season_Match_Data = pd.concat(match_data, ignore_index = True, keys = Game_name)
Season_Shot_Data = pd.concat(shot_data,ignore_index = True, keys = Game_name)

In [61]:
print(Season_Player_Data)

           Id Goals Own Goals Shots                    xG time Player ID  \
0      414500     0         0     0                     0   90      8704   
1      414501     0         0     1  0.020527511835098267   90      7077   
2      414502     0         0     0                     0   90      5274   
3      414503     0         0     0                     0   90      7184   
4      414504     0         0     0                     0   90      6834   
...       ...   ...       ...   ...                   ...  ...       ...   
10388  473498     0         0     1   0.36912643909454346   90      5595   
10389  473499     1         0     2    0.5518713593482971   90      9524   
10390  473502     0         0     0                     0    3      9558   
10391  473500     0         0     0                     0    1      9557   
10392  473501     0         0     0                     0    3      9359   

      Team Id Position      Player Name  ... Yellow Cards Red Cards Roster In  \
0     

In [62]:
print(Season_Shot_Data)

          ID Minute       Result             X_coord              Y_coord  \
0     376477      6    SavedShot               0.785  0.24899999618530275   
1     376481      9  BlockedShot  0.9330000305175781  0.34200000762939453   
2     376487     55  MissedShots  0.8130000305175781   0.3189999961853027   
3     376490     66    SavedShot  0.8030000305175782                0.655   
4     376494     89  MissedShots   0.595999984741211  0.29200000762939454   
...      ...    ...          ...                 ...                  ...   
9230  424161     42  MissedShots  0.8069999694824219                0.685   
9231  424162     43  MissedShots   0.754000015258789  0.29600000381469727   
9232  424163     48         Goal               0.885                  0.5   
9233  424166     57  MissedShots  0.8819999694824219                 0.44   
9234  424171     80    SavedShot  0.7719999694824219   0.6079999923706054   

                        xG          Player name Home/Away Player ID Situati

In [52]:
Game_data_2 = pd.concat([Game_Data, Season_Match_Data], axis = 1)

In [54]:
print(Game_data_2)

        id  Is Result?                Home Team          Away Team Home Goals  \
0    14086        True                   Fulham            Arsenal          0   
1    14087        True           Crystal Palace        Southampton          1   
2    14090        True                Liverpool              Leeds          4   
3    14091        True                 West Ham   Newcastle United          0   
4    14092        True     West Bromwich Albion          Leicester          0   
..     ...         ...                      ...                ...        ...   
375  14810        True                Liverpool     Crystal Palace          2   
376  14811        True          Manchester City            Everton          5   
377  14812        True         Sheffield United            Burnley          1   
378  14813        True                 West Ham        Southampton          3   
379  14814        True  Wolverhampton Wanderers  Manchester United          1   

    Away Goals   xG Home   

In [21]:
dfs_to_keep = []
for i in range(0, len(shot_data)):
    if i % 2 == 1:
        dfs_to_keep.append(shot_data[i])
Season_Shot_Data = pd.concat(dfs_to_keep, keys=Game_name)
Season_Shot_Data['Team Name'] = Season_Shot_Data['Team Id'].apply(id_to_name)

In [23]:
Season_Shot_Data.to_csv('EPL_Match_Data_2021.csv')

In [24]:
player_row_drop = []
for player in player_name:
    if (Season_Shot_Data['Player Name'][i] != player):
        player_row_drop.append([i])
player_df = Season_Shot_Data.drop(player_row_drop)
print(player_df)

NameError: name 'player_name' is not defined

In [62]:
player_row_drop = []
for i in range(0, len(list(Season_Shot_Data['Player Name']))):
    if (Season_Shot_Data['Player Name'][i] != 'Harry Kane'):
        player_row_drop.append(i)
        print(player_row_drop)
player_df = Season_Shot_Data.drop(player_row_drop)
print(player_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: 0